<a href="https://colab.research.google.com/github/sisco1113/LSTM-seq2seq-text-generation/blob/main/LSTM_seq2seq_text_generation_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

#Create int_to_int that changes integer to char.
int_to_char = {i: c for i, c in enumerate(chars)}

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
            seq_in = raw_text[i:i + seq_length]
            seq_out = raw_text[i + seq_length]
            dataX.append([char_to_int[char] for char in seq_in])
            dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
#print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nDone.")

Total Characters:  144343
Total Vocab:  44
Total Patterns:  144243
Epoch 1/5
1127/1127 [==============================] - ETA: 0s - loss: 2.8428
Epoch 1: loss improved from inf to 2.84280, saving model to weights-improvement-01-2.8428.keras
1127/1127 [==============================] - 38s 29ms/step - loss: 2.8428
Epoch 2/5
1127/1127 [==============================] - ETA: 0s - loss: 2.4858
Epoch 2: loss improved from 2.84280 to 2.48577, saving model to weights-improvement-02-2.4858.keras
1127/1127 [==============================] - 33s 30ms/step - loss: 2.4858
Epoch 3/5
1127/1127 [==============================] - ETA: 0s - loss: 2.2595
Epoch 3: loss improved from 2.48577 to 2.25949, saving model to weights-improvement-03-2.2595.keras
1127/1127 [==============================] - 33s 29ms/step - loss: 2.2595
Epoch 4/5
1127/1127 [==============================] - ETA: 0s - loss: 2.1105
Epoch 4: loss improved from 2.25949 to 2.11053, saving model to weights-improvement-04-2.1105.keras
112